In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("VectorStores")

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import csv

file_path = "../data/csv_data/rental_data.csv"


def get_csv_headers(file_path):
    with open(file_path, mode="r", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # 첫 번째 줄(헤더) 가져오기
    return headers


headers = get_csv_headers(file_path)

# CSV 로더 생성
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": headers,
    },
    # source_column="place",
    content_columns=headers,
    metadata_columns=["price", "place"],
)
docs = loader.load()
# print(docs[1])

In [ ]:
i = 1

for doc in docs[i:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>\n\n"

    docs[i].page_content = row_str
    i += 1
    # print(ret[i].page_content)
    # ret += row_str

ret = docs

In [ ]:
print(ret[1].metadata)

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 차원 크기를 계산
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

In [ ]:
# FAISS 벡터 저장소 생성
db = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# DB 생성
db = FAISS.from_documents(documents=ret[:3], embedding=OpenAIEmbeddings())

In [ ]:
# 벡터 저장소에 Document 문서 추가
from langchain_core.documents import Document

# page_content, metadata 지정
db.add_documents(ret[3:])

In [ ]:
# 문서 저장소 ID 확인
# db.index_to_docstore_id

# 저장된 내용
# db.docstore._dict

# id 로 삭제
# db.delete([id])

# 벡터 저장소를 병합
# db.merge_from(db2)

In [ ]:
# 임계값 기반 검색 수행
# retriever = db.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.7}
# )


# 메타데이터 필터 적용
faiss_retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
faiss_retriever.invoke("수정 오피스텔에 대해 알려줘")

In [ ]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    for doc in docs:
        print(doc.metadata["price"] + " " + doc.metadata["place"])

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# 로컬 Disk 에 저장
# db.save_local(folder_path="faiss_db", index_name="faiss_index")

# # 저장된 데이터를 로드
db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

# # 로드된 데이터를 확인
# loaded_db.index_to_docstore_id

In [ ]:
# # # ConfigurableField 클래스를 사용해 런타임에서 retriever 속성을 변경

# from langchain_core.runnables import ConfigurableField
# from langchain.retrievers import BM25Retriever, EnsembleRetriever

# # bm25 retriever와 faiss retriever를 초기화합니다.
# bm25_retriever = BM25Retriever.from_documents(
#     documents=ret[1:],
# )

# bm25_retriever.k=3

# ensemble_retriever = EnsembleRetriever(
#     # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
#     retrievers=[bm25_retriever, faiss_retriever],
# ).configurable_fields(
#     weights=ConfigurableField(
#         # 검색 매개변수의 고유 식별자를 설정합니다.
#         id="ensemble_weights",
#         # 검색 매개변수의 이름을 설정합니다.
#         name="Ensemble Weights",
#         # 검색 매개변수에 대한 설명을 작성합니다.
#         description="Ensemble Weights",
#     )
# )

# config = {"configurable": {"ensemble_weights": [0.7, 0.3]}}

# # config 매개변수를 사용하여 검색 설정을 지정합니다.
# docs = ensemble_retriever.invoke("중문 자취방 추천해줘", config=config)
# pretty_print_docs(docs)  # 검색 결과인 docs를 출력합니다.

In [ ]:
# self-query retriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="place",
        description="The location information of the rental room. One of ['정문', '중문', '후문', '기숙사 근처', '농가마트 근처', '교육문화회관 뒤']",
        type="string",
    ),
    AttributeInfo(
        name="price",
        description="The rental price of the room, which may include information about annual rent or half-year rent, or both",
        type="string",
    ),
]

In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a rental room",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

# StructuredQueryOutputParser 를 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성
query_constructor = prompt | llm | output_parser

In [ ]:
query_output = query_constructor.invoke(
    {
        # 쿼리 생성기를 호출하여 주어진 질문에 대한 쿼리를 생성합니다.
        "query": "중문 자취방 추천해주세요"
    }
)

In [ ]:
# # 쿼리 출력
# query_output.filter.arguments

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=db,  # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기
)

In [ ]:
# docs=retriever.invoke(
#     "후문 자취방 추천해주세요"
# )
# pretty_print_docs(docs)  # 검색 결과인 docs를 출력합니다.

In [ ]:
# retriever = SelfQueryRetriever.from_llm(
#     llm=llm,
#     vectorstore=db,
#     document_contents="Brief summary of a rental room",
#     metadata_field_info=metadata_field_info,
#     enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
#     search_kwargs={"k": 2},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
# )

In [ ]:
# # Self-query 검색
# retriever.invoke("2023년에 출시된 상품을 추천해주세요")